In [6]:
import google.generativeai as genai
import pandas as pd
import time
import os
import re
from tqdm import tqdm

# 👉 Configuration API Gemini
genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-2.5-flash-lite")  # version gratuite

# ---------------------------
# FONCTION : extraire les expressions avec Gemini
# ---------------------------
def extraire_expressions(text):
    prompt = f"""
    Analyse les commentaires suivants (issus d'avis clients pour un restaurant).
    Extrais des expressions clés ou des phrases courtes (2 à 4 mots maximum)
    qui décrivent le sentiment du client ou des aspects importants
    (qualité, service, ambiance, prix, etc.).
    Ne mets pas de motifs comme "**" autour des expressions dans ta réponse.
    Évite les phrases d'introduction ou de conclusion dans ta réponse.
    
    Pour chaque expression, indique une estimation de sa fréquence.
    Format de sortie :
    - expression: fréquence

    Commentaires :
    {text}
    """
    try:
        response = model.generate_content(prompt)
        time.sleep(5)  # temporisation API gratuite
        resultat = {}
        cleaned = re.sub(r"(voici.*?\n|\*\*.*?\*\*)", "", response.text, flags=re.IGNORECASE)

        for line in cleaned.strip().split("\n"):
            if ":" in line:
                expression, freq = line.split(":", 1)
                expression = expression.strip("- ").lower()
                freq_val = re.findall(r'\d+', freq)
                freq = int(freq_val[0]) if freq_val else 1
                if expression:
                    resultat[expression] = freq
        return resultat
    except Exception as e:
        print("Erreur Gemini:", e)
        return {}

# ---------------------------
# PARAMÈTRES
# ---------------------------
VILLE_CIBLE = "marakech"   # 👉 change ici la ville que tu veux analyser
TAILLE_BLOC = 2000      # Nombre max de caractères par bloc (évite de surcharger Gemini)

# ---------------------------
# LECTURE DES DONNÉES
# ---------------------------
df = pd.read_excel("marakech_V_finale.xlsx")
df["Commentaire_nettoye"] = df["Commentaire_nettoye"].fillna("").astype(str)
df = df[df["Commentaire_nettoye"].str.strip() != ""]

# Filtrer par ville choisie
df_ville = df[df["ville"].str.lower() == VILLE_CIBLE.lower()]

# Groupement des commentaires par restaurant
grouped = df_ville.groupby(["Nom du Restaurant", "ville"])["Commentaire_nettoye"].apply(lambda x: " ".join(x)).reset_index()

# ---------------------------
# ANALYSE
# ---------------------------
resultats = []

for _, row in tqdm(grouped.iterrows(), total=grouped.shape[0], desc=f"Analyse des restaurants de {VILLE_CIBLE}"):
    restaurant = row["Nom du Restaurant"]
    commentaires = row["Commentaire_nettoye"]

    # Découpage en blocs
    blocs = [commentaires[i:i+TAILLE_BLOC] for i in range(0, len(commentaires), TAILLE_BLOC)]

    # Dictionnaire global pour ce restaurant
    resultats_global = {}

    for bloc in blocs:
        expressions = extraire_expressions(bloc)
        for expr, freq in expressions.items():
            if expr in resultats_global:
                resultats_global[expr] += freq
            else:
                resultats_global[expr] = freq

    # Sauvegarde des résultats finaux pour ce resto
    for expr, freq in resultats_global.items():
        resultats.append({
            "restaurant": restaurant,
            "ville": VILLE_CIBLE,
            "expression": expr,
            "frequence": freq
        })

# ---------------------------
# EXPORT EXCEL
# ---------------------------
df_final = pd.DataFrame(resultats)
fichier_sortie = f"expressions_{VILLE_CIBLE}.xlsx"
df_final.to_excel(fichier_sortie, index=False)

print(f"✅ Analyse terminée pour {VILLE_CIBLE}. Résultats enregistrés dans {fichier_sortie}")


Analyse des restaurants de marakech:   6%|██▍                                       | 7/119 [19:16<4:49:49, 155.26s/it]

Erreur Gemini: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 4. Meaning that the model was reciting from copyrighted material.


Analyse des restaurants de marakech:  47%|██████████████████▎                    | 56/119 [2:51:55<5:00:32, 286.23s/it]

Erreur Gemini: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 1000
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 6
}
]
Erreur Gemini: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequest

Analyse des restaurants de marakech:  48%|██████████████████▋                    | 57/119 [2:56:48<4:57:58, 288.36s/it]

...

Analyse des restaurants de marakech:  99%|████████████████████████████████████████▋| 118/119 [2:57:58<00:01,  1.20s/it]

Erreur Gemini: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 1000
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 54
}
]
Erreur Gemini: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateReques

Analyse des restaurants de marakech: 100%|█████████████████████████████████████████| 119/119 [2:58:00<00:00, 89.75s/it]

Erreur Gemini: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 1000
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 53
}
]


✅ Analyse terminée pour marakech. Résultats enregistrés dans expressions_marakech.xlsx
